<a id="section_ID_0"></a>
# Workflow for prediction and analysis of assembly pathways of multi-protein complexes




## Table of Contents:

1. **[Requirements](#section_ID_1)**
 

1. **[Imports](#section_ID_2)**

1. **[Prepare Data and Run PTGLgraphComputation (Optional)](#section_ID_3)**
    <br>3.1 [Define Options for PTGL](#section_ID_3.3)
    <br>3.2 [Run PTGL](#section_ID_3.4)

1. **[Implementation of assembly pathway preditction workflow](#section_ID_4)**
    <br>4.1 [Method to process the graph to find all labels](#section_ID_4.2)
    <br>4.2 [Methods for newick string conversion](#section_ID_4.3)
    <br>4.3 [Methods for creation of distance matrix](#section_ID_4.4)
    <br>4.4 [Cluster algorithms](#section_ID_4.5)
        <br>4.4.1 [Ward linkage clustering](#section_ID_4.5.1)
        <br>4.4.2 [Weighted linkage clustering](#section_ID_4.5.2)
        <br>4.4.3 [UPGMA clustering](#section_ID_4.5.3)
        <br>4.4.4 [Single linkage clustering](#section_ID_4.5.4)
        <br>4.4.5 [Weighted Linkage clustering](#section_ID_4.5.5)
    <br>4.5 [Preprocessing](#section_ID_4.6)
    <br>4.6 [Clustering](#section_ID_4.6)


 
<a id="section_ID_1"></a>
## 1. Requirements

- igraph
- scipy
- numpy
- matplot
- We used PTGL (optional) for the graph computation:  
    - DSSP files (can be loaded from https://pdb-redo.eu/dssp)
    -  .cif file of the multi-protein complex 

<a id="section_ID_2"></a>
## 2. Imports

In [ ]:
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform
import igraph as ig
import matplotlib.pyplot as plt
import numpy as np
# Optional
from io import StringIO 
import os 
from generalFunctions import download_pdb, createLayout, modifyLayoutPosition
import shutil
import pickle
import warnings
import subprocess

<a id="section_ID_3"></a>
## 3. Prepare data and run PTGLgraphComputation (optional)

<div class="alert alert-block alert-info"><b>This example notebook is provided with the required files for all steps and will also run if no PTGL software is available.</b><br>Set variables to False to run the example without installing PTGL, and DSSP software. Set to True and provide the paths if the software is available. In this case procede with the protein graph at 4.1</div>

<a id="section_ID_3.3"></a>
### 3.1 Define options for PTGL

use pdb_id to set the pdb_id of the structure of interesst 

In [ ]:
pdb_id = "here id"
base_dir = os.getcwd()

# Zielverzeichnis: \PTGLtools\PTGLgraphComputation\dist
ptgl_dir = os.path.join(base_dir, 'PTGLtools', 'PTGLgraphComputation', 'dist')
output_dir = os.path.join(base_dir, "ptgl_output")
output_dir
    # Definiere den Befehl
command = [
        "java", 
        "-jar", 
        "PTGLgraphComputation.jar", 
        pdb_id, 
        "-d",
        pdb_id+".cif",
        "-G", 
        "-o",
        output_dir,
        "--silent"
    ]



<a id="section_ID_3.4"></a>
### 3.2 Run PTGL

In [ ]:
try:
    # Wechsle ins Zielverzeichnis und führe den Befehl aus
    result = subprocess.run(command, cwd=ptgl_dir, capture_output=True, text=True)
    # Gib die Ausgabe des Befehls aus
    print("Kommando-Ausgabe:")
    print(result.stdout)
    print("Fehler (falls vorhanden):")
    print(result.stderr)
except Exception as e:
    print(f"Ein Fehler ist aufgetreten: {e}")

<a id="section_ID_4"></a>
## 4. Implementation of the assembly pathway prediction workflow 

<a id="section_ID_4.2"></a>
### 4.1 Method to process the graph to find all labels

In [ ]:
graph_file: str = f"{output_dir}/"+pdb_id+"_complex_chains_albelig_CG.gml"
complex_graph = ig.Graph.Read_GML(graph_file)
ig.summary(complex_graph)

In [ ]:
def get_labels(complex_graph):
    """
    Finds all labels for the vertices

    :param complex_graph: graph file in gml format
    :returns: list with labels
    """
    nodes = complex_graph.vs
    labels = nodes["label"]
    return labels

<a id="section_ID_4.3"></a>
### 4.2 Methods for newick string conversion

In [ ]:
def get_newick(node, parent_dist, leaf_names):
    """
    Convert sciply.cluster.hierarchy.to_tree()-output to Newick format.

    :param node: output of sciply.cluster.hierarchy.to_tree()
    :param parent_dist: output of sciply.cluster.hierarchy.to_tree().dist
    :param leaf_names: list of leaf names
    :returns: tree in Newick format
    """
    if node.is_leaf():
        return leaf_names[node.id]
    left_newick = get_newick(node.get_left(), node.dist, leaf_names)
    right_newick = get_newick(node.get_right(), node.dist, leaf_names)
    return f"({left_newick},{right_newick}):{parent_dist}"


def newick_parameters(linkage_matrix, labels):
    """ 
    Convert a given linkage matrix from scipy to a newick string
    
    :param: linkage_matrix
    :returns: linkage matrix as newick string 
    """
    tree = sch.to_tree(linkage_matrix)
    newick = get_newick(tree, tree.dist, labels)
    return f"({newick});"

<a id="section_ID_4.4"></a>
### 4.3 Method for creation of distance matrix

In [ ]:
def create_distance_matrix(complex_graph):
    """ 
    Reads in the complex graph and returns the condensed distance matrix 
    
    :param: complex_graph: graph saved as object from igraph libary 
    :returns: condensed_matrix 
    """
    adj_matrix_absw = complex_graph.get_adjacency(attribute="absoluteWeight")
    adj_array_absw_list = np.array(adj_matrix_absw.data)
    # Calculates the distance matrix
    max_value = np.max(adj_array_absw_list)
    dist_matrix = -1 * (adj_array_absw_list - max_value) / max_value
    np.fill_diagonal(dist_matrix, 0)
    # Calculates the condensed distance matrix
    condensed_matrix = squareform(dist_matrix)
    print(condensed_matrix)
    return condensed_matrix
    

<a id="section_ID_4.5"></a>
### 4.4 Cluster algorithms

<a id="section_ID_4.5.1"></a>
#### 4.4.1 Ward linkage Clustering 

In [ ]:
def ward_linkage_clustering(condensed_matrix):
    """ 
    Reads in the condensed distance matrix and creates tthe tre clustered with ward-linkage
    
    :param: condensed_matrix: condensed distance matrix saved as numpy array 
    :returns: ns_ward_absw: contains the tree as newick string  
    """
    ward_linkage_matrix = sch.linkage(condensed_matrix, method="ward", metric='euclidean', optimal_ordering=True)

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle("absolute weight ward linkage", fontsize=16)
    dn = sch.dendrogram(ward_linkage_matrix, labels = labels)
    ns_ward_absw = newick_parameters(ward_linkage_matrix, labels)
    return ns_ward_absw


#### <a id="section_ID_4.5.2"></a>
#### 4.4.2 Weighted linkage Clustering

In [ ]:
def weighted_linkage_clustering(condensed_matrix):
    """ 
    Reads in the condensed distance matrix and creates the tree clusterd with weighted-linkage 
    
    :param: condensed_matrix: condensed distance matrix saved as numpy array 
    :returns: ns_weighted_absw: contains the tree as newick string  
    """
    weighted_linkage_matrix = sch.linkage(condensed_matrix, method="weighted", metric='euclidean', optimal_ordering=True)
    
    fig= plt.figure(figsize=(10, 5))
    #plt.grid(axis='y')
    #plt.yticks(np.arange(0, 1.1, 0.05))
    fig.suptitle("absolute weighted linkage", fontsize=16)
    dn = sch.dendrogram(weighted_linkage_matrix, labels = labels)
    ns_weighted_absw = newick_parameters(weighted_linkage_matrix, labels)
    return ns_weighted_absw


<a id="section_ID_4.5.3"></a>
#### 4.4.3 UPGMA linkage Clustering

In [ ]:
def UPGMA_linkage_clustering(condensed_matrix):
    """ 
    Reads in the condensed distance matrix and creates the tree clustered with UPGMA
    
    :param: condensed_matrix: condensed distance matrix saved as numpy array 
    :returns: ns_UPGMA_absw: contains the tree as newick string  
    """
    UPGMA_linkage_matrix = sch.linkage(condensed_matrix, method="average", metric='euclidean', optimal_ordering=True)

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle("absolute weight UPGMA", fontsize=16)
    dn = sch.dendrogram(UPGMA_linkage_matrix, labels = labels)
    ns_UPGMA_absw = newick_parameters(UPGMA_linkage_matrix, labels)
    return ns_UPGMA_absw

<a id="section_ID_4.5.4"></a>
#### 4.4.4 Single linkage Clustering

In [ ]:
def single_linkage_clustering(condensed_matrix):
    """ 
    Reads in the condensed distance matrix and creates the tree clustered with single-linkage
    
    :param: condensed_matrix: condensed distance matrix saved as numpy array 
    :returns: ns_single_absw: contains the tree as newick string  
    """
    single_linkage_matrix = sch.linkage(condensed_matrix, method="single", metric='euclidean', optimal_ordering=True)

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle("absolute single linkage", fontsize=16)
    dn = sch.dendrogram(single_linkage_matrix, labels = labels)
    ns_single_absw = newick_parameters(single_linkage_matrix, labels)
    return ns_single_absw

<a id="section_ID_4.5.5"></a>
#### 4.4.5 Complete linkage Clustering

In [ ]:
def complete_linkage_clustering(condensed_matrix):
    """ 
    Reads in the condensed distance matrix and creates the tree clustered with UPGMA
    
    :param: condensed_matrix: condensed distance matrix saved as numpy array 
    :returns: ns_complete_absw: contains the tree as newick string  
    """
    complete_linkage_matrix = sch.linkage(condensed_matrix, method="complete", metric='euclidean', optimal_ordering=True)

    fig = plt.figure(figsize=(10, 5))
    fig.suptitle("absolute complete linkage", fontsize=16)
    dn = sch.dendrogram(complete_linkage_matrix, labels = labels)
    ns_complete_absw = newick_parameters(complete_linkage_matrix, labels)
    return ns_complete_absw

<a id="section_ID_4.6"></a>
### 4.5 Preprocessing


In [ ]:
# uses method get_labels to save the labels in list labels
labels = get_labels(complex_graph)

# uses method create_distance_matrix() to create the condesned distance matrix and saves as numpay array condensed_matrix
condensed_matrix = create_distance_matrix(complex_graph)

<a id="section_ID_4.7"></a>
### 4.6 Clustering

<div class="alert alert-block alert-info"><b>This notebook contains different clustering algorithms to calculate different possible assembling pathways.</b><br>While testing weighted-linkage has proven to bring the most accurate results.</div>

In [ ]:
# uses method ward_linkage_custering() to cluster the matrix and saves the clustering as newick string 
ward_linkage = ward_linkage_clustering(condensed_matrix)

# uses method wweighted_linkage_clustering() to cluster the matrix and saves the clustering as newick string 
weighted_linkage = weighted_linkage_clustering(condensed_matrix)

# uses method UPGMA_linkage_clustering() to cluster the matrix and saves the clustering as newick string 
UPGMA_linkage = UPGMA_linkage_clustering(condensed_matrix)

# uses method single_linkage_clustering() to cluster the matrix and saves the clustering as newick string 
single_linkage = single_linkage_clustering(condensed_matrix)

# uses method complete_linkage_clustering() to cluster the matrix and saves the clustering as newick string 
complete_linkage = complete_linkage_clustering(condensed_matrix)

In [ ]:
UPGMA_linkage

In [ ]:
weighted_linkage

In [ ]:
ward_linkage

In [ ]:
single_linkage

In [ ]:
complete_linkage